In [1]:
import pandas as pd
import requests 
import lxml.html
import os


In [4]:
# Get All Events
# https://eurovision.tv/event/tel-aviv-2019/grand-final/results/albania

from fake_useragent import UserAgent
ua = UserAgent()

url = 'https://eurovision.tv/events/'
r = requests.get( url , headers = {'User-Agent': ua.random}  )
html = lxml.html.fromstring( r.text )
events = html.xpath('//div[@class="flex flex-wrap"]/div/div/a')

event_links = [i.get('href') for i in events]
df_events = pd.DataFrame( event_links , columns= ['url'])

df_events['year'] = [ int(r.split('-')[ len(r.split('-'))-1 ]) for i,r in df_events['url'].iteritems() ]
df_events['place'] = [ r.split('/')[ len(r.split('/'))-1 ] for i,r in df_events['url'].iteritems() ]
df_events['place'] = [ " ".join(r.split('-')[ :len(r.split('-'))-1 ]) for i,r in df_events['place'].iteritems() ]

df_events

,url,year,place
0,https://eurovision.tv/event/rotterdam-2021,2021,rotterdam
1,https://eurovision.tv/event/rotterdam-2020,2020,rotterdam
2,https://eurovision.tv/event/tel-aviv-2019,2019,tel aviv
3,https://eurovision.tv/event/lisbon-2018,2018,lisbon
4,https://eurovision.tv/event/kyiv-2017,2017,kyiv
...,...,...,...
61,https://eurovision.tv/event/london-1960,1960,london
62,https://eurovision.tv/event/cannes-1959,1959,cannes
63,https://eurovision.tv/event/hilversum-1958,1958,hilversum
64,https://eurovision.tv/event/frankfurt-1957,1957,frankfurt


In [47]:
import time
import random
#Extract Data

def get_pts( df , year ): #Get given years votes per country

    from fake_useragent import UserAgent
    ua = UserAgent()

    all_pts = pd.DataFrame()
    #Format changes in 2004 - Istanbul

    if year < 2004: url = df[ df['year']==year ]['url'].tolist()[0] + '/final/results/'
    if year >= 2004: url = df[ df['year']==year ]['url'].tolist()[0] + '/grand-final/results/'
    #Get all countries for results
    r = requests.get( url , headers = {'User-Agent': ua.random}  )
    html = lxml.html.fromstring( r.text )
    countries = html.xpath('//option')
    country_links = [i.get('value') for i in countries] 

    proxies = {
        'http': 'http://135.181.68.88:3128',
        'https': 'http://135.181.68.88:3128',
    }

    for c in country_links: #Get points received
        if c:
            try:
                r = requests.get( c ,
                    headers = {'User-Agent': ua.random} ,
                    #proxies= proxies
                    )
                pts = pd.read_html( r.text )[0]
                pts.columns = ['pts','pts_from']
                pts['year'] = year
                pts['points_to'] = c.split('/')[len(c.split('/'))-1] 
                all_pts = all_pts.append( pts )
            except:
                print( c )
        time.sleep( random.random()*3 )
    if len(all_pts) == 0: print( str(year), 'is empty' )
    return all_pts


In [59]:
year = 2018
folder = r'C:\Users\csucuogl\Documents\GitHub\IdeasTesting\Songs\data'
temp = get_pts( df_events , year )

temp.to_csv( os.path.join(folder,'results_'+str(year)+'.csv' ) )

temp

,pts,pts_from,year,points_to
0,12,Albania,2018,italy
1,12,Malta,2018,italy
2,12,Germany,2018,italy
3,10,France,2018,italy
4,10,Slovenia,2018,italy
...,...,...,...,...
9,1,United Kingdom,2018,san-marino
0,6,Malta,2018,australia
1,2,Denmark,2018,australia
2,1,United Kingdom,2018,australia


In [4]:
folder = r'C:\Users\csucuogl\Documents\GitHub\IdeasTesting\Songs\data' 
files = os.listdir( folder )
files = [ i for i in files if "results" in i]

files

['results_1958.csv',
 'results_1959.csv',
 'results_1960.csv',
 'results_1961.csv',
 'results_1962.csv',
 'results_1963.csv',
 'results_1964.csv',
 'results_1965.csv',
 'results_1966.csv',
 'results_1967.csv',
 'results_1968.csv',
 'results_1969.csv',
 'results_1970.csv',
 'results_1971.csv',
 'results_1972.csv',
 'results_1973.csv',
 'results_1974.csv',
 'results_1975.csv',
 'results_1976.csv',
 'results_1977.csv',
 'results_1978.csv',
 'results_1979.csv',
 'results_1980.csv',
 'results_1981.csv',
 'results_1982.csv',
 'results_1983.csv',
 'results_1984.csv',
 'results_1985.csv',
 'results_1986.csv',
 'results_1987.csv',
 'results_1988.csv',
 'results_1989.csv',
 'results_1990.csv',
 'results_1991.csv',
 'results_1992.csv',
 'results_1993.csv',
 'results_1994.csv',
 'results_1995.csv',
 'results_1996.csv',
 'results_1997.csv',
 'results_1998.csv',
 'results_1999.csv',
 'results_2000.csv',
 'results_2001.csv',
 'results_2002.csv',
 'results_2003.csv',
 'results_2004.csv',
 'results_200

In [20]:
import pandas as pd

df = pd.DataFrame()
for f in files:
    temp = pd.read_csv(os.path.join(folder,f))
    df = df.append( temp )




df['points_to'] = df['points_to'].str.replace("-",' ')
df['points_to'] = [r.title() for i,r in df['points_to'].iteritems() ]
df = df[['pts',"pts_from",'year','points_to']]
df = df[ df['pts_from'] != 'points received' ]
df.head( 15 )

,pts,pts_from,year,points_to
0,1,France,1958,Denmark
1,1,The Netherlands,1958,Denmark
2,1,Sweden,1958,Denmark
4,2,France,1958,Germany
5,1,Austria,1958,Germany
6,1,Belgium,1958,Germany
7,1,Sweden,1958,Germany
9,6,The Netherlands,1958,Switzerland
10,5,Luxembourg,1958,Switzerland
11,4,Italy,1958,Switzerland


In [26]:
df[ df['pts_from'].str.contains("&") ]
df['pts_from'] = df['pts_from'].str.replace("&","")

,pts,pts_from,year,points_to
11,8,Bosnia & Herzegovina,1993,Ireland
78,10,Bosnia & Herzegovina,1993,The Netherlands
92,2,Bosnia & Herzegovina,1993,Turkey
131,1,Bosnia & Herzegovina,1993,Slovenia
188,3,Bosnia & Herzegovina,1993,United Kingdom
...,...,...,...,...
425,7,Bosnia & Herzegovina,2016,Ukraine
492,2,Bosnia & Herzegovina,2016,Bulgaria
510,6,Bosnia & Herzegovina,2016,Czech Republic
555,12,Bosnia & Herzegovina,2016,Serbia


In [22]:
df.to_csv(r"C:\Users\csucuogl\Documents\GitHub\IdeasTesting\Songs\data\all_years.csv")